In [ ]:
# Importing relevant libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
import pandas as pd
import yfinance as yf

In [ ]:
# Reading in the base input dataset
df = pd.read_excel('D:\SDG2000-shareable-list_28.04.2022_initial_data.xlsx', sheet_name = '2. SDG2000')

In [ ]:
df['SEDOL Code'] = df['SEDOL Code'].fillna(0) # filling any missing SEDOL code companies with 0s
df = df.loc[df['SEDOL Code'] != 0] # filtering out the missing SEDOL code companies
df = df.reset_index(drop = True) # Reseting the index of the dataframe
df['Ticker'] = '' # Adding a new blank column where the scraped tickers can be stored

In [ ]:
for (index, Sedol) in enumerate(df['SEDOL Code'][0:1444]): 
    
    company_sedol = Sedol # Creating a new variable for a specific company name
    options = Options() # Creating advanced options for the Chrome webdriver
    ua = UserAgent() # Creating useragents as part of the protocol to prevent scraper being detected
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}') # Adding in the randomly generated useragent
    options.add_experimental_option('useAutomationExtension', False) # Additional tools to prevent detection
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_argument('--disable-blink-features=AutomationControlled')

    driver = webdriver.Chrome('C:\chromedriver.exe',options=options) # Launching a webdriver with the options
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 
    
    driver.get('https://finance.yahoo.com/') # Yahoo Finance website

    wait = WebDriverWait(driver,5) # Defining wait variable for later use
    action = ActionChains(driver) # Defining action variable for later use
    
    # Cookies, this chunk first goes into more options, clicks reject all, and then clicks confirm
    wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div/div/div/form/div[2]/div[2]/a')))
    elem = driver.find_element(By.XPATH, '/html/body/div/div/div/div/form/div[2]/div[2]/a')
    elem.click()
    wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div/form/div[2]/div[3]/div[2]/div/div/div[2]/div[2]/div[2]/label/span[2]')))
    elem = driver.find_element(By.XPATH, '/html/body/div/div/form/div[2]/div[3]/div[2]/div/div/div[2]/div[2]/div[2]/label/span[2]')
    elem.click()
    wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div/form/div[3]/div/div/button')))
    elem = driver.find_element(By.XPATH, '/html/body/div/div/form/div[3]/div/div/button')
    elem.click()
    
    
    
    wait.until(EC.presence_of_element_located((By.XPATH,'/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/input[1]' )))
    elem = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/input[1]') # Finding the search bar
    try:
        elem.clear() # Clearing any default values typed into the search bar
        elem.send_keys(company_sedol) # Typing in the company SEDOL code in the search bar 
        wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/div[1]/button')))
        elem = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/div[1]/button')
        elem.click()
        
        searchTextbox = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[6]/div/div/div/div[2]/div[1]/div[1]/h1'))) # Waiting
        # until the suggested company page shows up after searching with the SEDOL code.

    
        # Find the name of company ticker on the page and add it to the ticker column
        company_ticker_yahoo_scraper = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[6]/div/div/div/div[2]/div[1]/div[1]/h1')
        company_ticker_yahoo = company_ticker_yahoo_scraper.text
        df.iat[index,14] = (company_ticker_yahoo.split("(")[-1]).split(")")[0]
        
        driver.close() # Closing the driver
    # If the company is not found by SEDOL code, then the ISIN is used to try and get the company ticker    
    except TimeoutException:
        try:
            company_ISIN = df['ISIN'][index]
            driver.get('https://finance.yahoo.com/')
            wait.until(EC.presence_of_element_located((By.XPATH,'/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/input[1]' )))
            elem = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/input[1]')
            elem.send_keys(company_sedol) # Typing in the company SEDOL code in the search bar 
            wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/div[1]/button')))
            elem = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div/div/div[1]/div/div[2]/div/form/div[1]/button')
            elem.click()
        
            searchTextbox = wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[6]/div/div/div/div[2]/div[1]/div[1]/h1'))) # Waiting
            # until the suggested company page shows up after searching with the SEDOL code.

    
            # Find the name of company ticker on the page and add it to the ticker column
            company_ticker_yahoo_scraper = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[6]/div/div/div/div[2]/div[1]/div[1]/h1')
            company_ticker_yahoo = company_ticker_yahoo_scraper.text
            df.iat[index,14] = (company_ticker_yahoo.split("(")[-1]).split(")")[0]
        
            driver.close() # Closing the driver 
        except TimeoutException as ex: # If the company name is not found and a timeout error is raised,
            # add NotFound to the Ticker column for the current company
            isrunning = 0
            df.iloc[index,14] = 'NotFound'
            driver.close()
        

In [ ]:
df.to_excel('SDG2000_Yahoo_Tickers.xlsx') # exporting the scraped database as an excel

In [ ]:
tickers = (df['Ticker'].tolist())[0:1000]

# Retrieve Yahoo! Finance Sustainability metrics for each ticker
esg = pd.DataFrame()
for i in tickers:
    print(i)
    i_y = yf.Ticker(i)
    try:
        if i_y.sustainability is not None:
            temp = pd.DataFrame.transpose(i_y.sustainability)
            temp['company_ticker'] = str(i_y.ticker)
            # print(temp)
            esg = esg.append(temp)
    except IndexError:
        pass

In [ ]:
esg.to_excel('SDG2000_Yahoo_Sustainability.xlsx') # exporting the scraped database as an excel

In [ ]:
tickers = (df['Ticker'].tolist())[0:1000]

# Retrieve Yahoo! Finance financial metrics for each ticker
company_metrics = {}
for company in tickers:
    company_metrics[company] = {}
    company_info = yf.Ticker(company)
    try:
        if company_info.info is not None:
            company_metrics[company]['Profit_Margins'] = company_info.info['profitMargins']
            company_metrics[company]['Market_Cap'] = company_info.info['marketCap']
            company_metrics[company]['Total_Revenue'] = company_info.info['totalRevenue']
            company_metrics[company]['Enterprise_to_Revenue'] = company_info.info['enterpriseToRevenue']
            company_metrics[company]['Price_To_Sales_Trailing12Months'] = company_info.info['priceToSalesTrailing12Months']
            company_metrics[company]['Beta'] = company_info.info['beta']
    except:
        pass
finance = pd.DataFrame.from_dict(company_metrics)

In [ ]:
finance.to_excel('SDG2000_Yahoo_Financial.xlsx') # exporting the scraped database as an excel